# Collecting data

In [ ]:
# План простой:
# 1. собрать данные, возможно, что не такая глубина, как у автора статьи, завести бейзлайн с тем же риск-менеджментом,
# что и у автора статьи, прогнать его по данным, посмотреть на результаты.
# 2. Посмотреть как стратегия работает на споте, на фьючерсе.

# 3. прогнать на том, что есть с HL по фьючерсам, споту (мало данных)
# 4. посмотреть на результаты в тестнете

# 5. добавить ончейн данные и все рекомендации автора статьи

In [1]:
import pandas as pd

binance_eth_data_path = "/Users/a.tikhonov/ml_dl_projects/applied_ai_blockchain/data/1d_timeframe/spot/binance_spot_ETH_USDT_1d_20150425_20250425.csv"
binance_perpetual_data_path = "/Users/a.tikhonov/ml_dl_projects/applied_ai_blockchain/data/1d_timeframe/perps/binance_ETH_USDT:USDT_1d_20150425_20250425.csv"

binance_eth_spot_data = pd.read_csv(binance_eth_data_path)
binance_eth_perpetual_data = pd.read_csv(binance_perpetual_data_path)

In [75]:
hl_data = pd.read_csv("/Users/a.tikhonov/ml_dl_projects/applied_ai_blockchain/data/1d_timeframe/perps/SOL_USDC_USDC_hyperliquid_1d_20200914_to_20250425.csv")

In [86]:
import pandas_ta as ta

donchian_features = ta.donchian(high=hl_data["close"], low=hl_data["close"], length=5, lower_length=5, upper_length=5)
print(donchian_features.columns)

Index(['DCL_5_5', 'DCM_5_5', 'DCU_5_5'], dtype='object')


In [80]:
donchian_features


,DCL_20_20,DCM_20_20,DCU_20_20
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
1680,105.24,122.510,139.78
1681,105.24,126.930,148.62
1682,105.24,128.145,151.05
1683,105.24,128.885,152.53


In [73]:
hl_eth_data.rename(columns={"datetime": "timestamp"}, inplace=True)


In [74]:
hl_eth_data.to_csv("/Users/a.tikhonov/ml_dl_projects/applied_ai_blockchain/data/1d_timeframe/perps/SOL_USDC_USDC_hyperliquid_1d_20200914_to_20250425.csv", index=False)

In [2]:
binance_eth_spot_data


,timestamp,open,high,low,close,volume
0,2017-08-17 00:00:00+00:00,301.13,312.18,298.00,302.00,7.030710e+03
1,2017-08-18 00:00:00+00:00,302.00,311.79,283.94,293.96,9.537846e+03
2,2017-08-19 00:00:00+00:00,293.31,299.90,278.00,290.91,2.146198e+03
3,2017-08-20 00:00:00+00:00,289.41,300.53,282.85,299.10,2.510139e+03
4,2017-08-21 00:00:00+00:00,299.10,346.52,294.60,323.29,5.219445e+03
...,...,...,...,...,...,...
2804,2025-04-21 00:00:00+00:00,1587.36,1658.75,1564.07,1579.57,6.914867e+05
2805,2025-04-22 00:00:00+00:00,1579.57,1778.00,1537.26,1756.26,1.103037e+06
2806,2025-04-23 00:00:00+00:00,1756.25,1834.86,1744.95,1795.07,8.875456e+05
2807,2025-04-24 00:00:00+00:00,1795.08,1802.82,1722.90,1769.65,4.834650e+05


# Entry and exit criteria

A trend-following system is constructed such that as soon as the closing price hits the
upper boundary of the Donchian Channel, a new long position is established. Each
position is then maintained until the first closure below a trailing stop which is updated
daily according to the following formula

In [26]:
import pandas_ta as ta

donchian_features = ta.donchian(
    high=binance_eth_spot_data["close"],
    low=binance_eth_spot_data["close"],
    length=20
)



In [29]:
eth_spot_data = pd.concat([binance_eth_spot_data, donchian_features], axis=1)


In [54]:
def get_trailing_stop_feature(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["trailing_stop"] = df["DCM_20_20"].copy()
    for i in range(1, len(df)):
        df.loc[i, "trailing_stop"] = max(
            df.loc[i - 1, "DCM_20_20"], df.loc[i - 1, "trailing_stop"]
        )
    return df


eth_spot_data = get_trailing_stop_feature(eth_spot_data)

# Postition sizing

In [62]:
strategy = pd.read_csv("/Users/a.tikhonov/ml_dl_projects/applied_ai_blockchain/data/1d_timeframe/spot/binance_spot_ETH_USDT_1d_20150425_20250425_strategy_output.csv")

In [63]:
strategy

,timestamp,open,high,low,close,volume,sigma_t,DCL_5,DCM_5,DCU_5,...,DCM_360,DCU_360,Pos_360,TrailingStop_360,w_360,w_Combo,asset_return,strategy_return,cumulative_asset_return,cumulative_strategy_return
0,2017-08-17 00:00:00+00:00,301.13,312.18,298.00,302.00,7.030710e+03,0.914372,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,2017-08-18 00:00:00+00:00,302.00,311.79,283.94,293.96,9.537846e+03,0.914372,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,0.0,0.0,-0.026623,-0.0,0.973377,1.000000
2,2017-08-19 00:00:00+00:00,293.31,299.90,278.00,290.91,2.146198e+03,0.914372,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,0.0,0.0,-0.010376,-0.0,0.963278,1.000000
3,2017-08-20 00:00:00+00:00,289.41,300.53,282.85,299.10,2.510139e+03,0.914372,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,0.0,0.0,0.028153,0.0,0.990397,1.000000
4,2017-08-21 00:00:00+00:00,299.10,346.52,294.60,323.29,5.219445e+03,0.914372,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,0.0,0.0,0.080876,0.0,1.070497,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2804,2025-04-21 00:00:00+00:00,1587.36,1658.75,1564.07,1579.57,6.914867e+05,0.699158,1473.41,1645.32,1817.23,...,1645.32,1817.23,0.0,NaN,0.0,0.0,-0.004908,-0.0,5.230364,4.811217
2805,2025-04-22 00:00:00+00:00,1579.57,1778.00,1537.26,1756.26,1.103037e+06,0.724154,1473.41,1645.32,1817.23,...,1645.32,1817.23,0.0,NaN,0.0,0.0,0.111860,0.0,5.815430,4.811217
2806,2025-04-23 00:00:00+00:00,1756.25,1834.86,1744.95,1795.07,8.875456e+05,0.723244,1473.41,1645.14,1816.87,...,1645.14,1816.87,0.0,NaN,0.0,0.0,0.022098,0.0,5.943940,4.811217
2807,2025-04-24 00:00:00+00:00,1795.08,1802.82,1722.90,1769.65,4.834650e+05,0.723344,1473.41,1639.71,1806.01,...,1639.71,1806.01,0.0,NaN,0.0,0.0,-0.014161,-0.0,5.859768,4.811217
